In [1]:
%pip install numpy pandas pillow tensorflow opencv-python matplotlib

  Using cached tensorflow-2.18.0-cp39-cp39-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp39-cp39-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.3-cp39-cp39-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.8.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached h5py-3.12.1-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.4.1-cp39-cp39-win_amd64.whl.metadata (20 kB)
  Using cached tens

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# 1. Load Pre-trained MobileNetV2 Model (Global)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D()  # Flatten the features 
])

# 2. Preprocess Input Image (User's Image)
def preprocess_input_image(img_path):
    img = Image.open(img_path)  # Open the image from the provided path
    img = img.resize((224, 224))  # Resize image to match MobileNetV2 input
    img = np.array(img)
    if img.shape[2] == 4:  # Handle images with transparency (alpha channel)
        img = img[:, :, :3]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Preprocess for MobileNetV2 (scaling pixel values)
    return img

# 3. Extract Features from a Single Image (User Image or Dataset Image)
def extract_features(img_path):
    img = Image.open(img_path)  # Open the image from the provided path
    img = img.resize((224, 224))  # Resize image to match MobileNetV2 input size

    # Check if the image has an alpha channel (RGBA), and remove it if present
    if img.mode == 'RGBA':
        img = img.convert('RGB')  # Convert to RGB by discarding the alpha channel

    img = np.array(img)  # Convert the image to a NumPy array
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = img.astype('float32')  # Convert the image to float32 type
    img = img / 255.0  # Normalize the image

    # Extract features using MobileNetV2 model
    features = model.predict(img)
    return features.flatten()  # Flatten to make it 1D vector

# 4. Extract Features from Dataset Images (Handling Directories and Files)
def extract_features_from_dataset(dataset_dir):
    features_list = []
    image_paths = []
    
    # Use os.walk to handle subdirectories and files
    for root, dirs, files in os.walk(dataset_dir):  # os.walk handles subdirectories
        for img_name in files:
            # Only process image files (check if it's a valid image extension)
            if img_name.lower().endswith(('jpg', 'jpeg', 'png', 'bmp')):
                img_path = os.path.join(root, img_name)
                try:
                    features = extract_features(img_path)
                    features_list.append(features)
                    image_paths.append(img_path)
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")
    
    return np.array(features_list), image_paths

# 5. Compute Similarity
def get_most_similar_images(user_img_path, dataset_dir, top_n=5):
    # Step 1: Extract features for the user image
    user_features = extract_features(user_img_path)

    # Step 2: Extract features from the dataset
    dataset_features, image_paths = extract_features_from_dataset(dataset_dir)

    # Step 3: Calculate similarity between user image and dataset images using cosine similarity
    similarities = cosine_similarity([user_features], dataset_features)[0]

    # Step 4: Get top N most similar images
    most_similar_idx = np.argsort(similarities)[-top_n:][::-1]
    similar_images = [image_paths[idx] for idx in most_similar_idx]

    return similar_images, similarities[most_similar_idx]

# 6. Display the Results
def display_similar_images(user_img_path, similar_images, similarities):
    # Display the user image
    img = Image.open(user_img_path)
    plt.figure(figsize=(12, 12))
    plt.subplot(3, 3, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title("User Image")

    # Display the most similar images
    for i, (sim_img, similarity) in enumerate(zip(similar_images, similarities)):
        img = Image.open(sim_img)
        plt.subplot(3, 3, i+2)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Similarity: {similarity:.2f}")

    plt.show()

# Example usage:
user_img_path = '/kaggle/input/testing-imageskeshawa/womanSkirt.png'  # Path to the user's uploaded image
dataset_dir = '/kaggle/input/moddelite/ModdeDataset/skirt'  # Path to your fashion dataset

# Step 1: Get the most similar imagesssss
similar_images, similarities = get_most_similar_images(user_img_path, dataset_dir, top_n=5)

# Step 2: Display the results
display_similar_images(user_img_path, similar_images, similarities)


In [ ]:
print("nana")